# Machine Learning Implementation

## Imports

In [1]:
import itertools
import json
import logging

import graphviz
import numpy as np
import pandas as pd
import plotly.offline as py
from graphviz import Digraph
from IPython.display import display
from plotly import graph_objects as go
from scipy.special import logsumexp, expit
from sklearn.datasets import load_boston, load_iris
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import export_graphviz

## Neural network

### The maths

A neural network mimics the brain as a series of nodes structured in layers. The network passes information between the layers depending on the layer's weights and the outputs from the previous layer.

Let's assume we have data $x_1, x_2, \dots x_m \in \mathbb{R}^{(n^0,1)}$ where m is the number of training examples and $n^0$ is the number of features in $x_i$
Let's assume a classification problem with $y_1, y_2, \dots y_m \in{0,1}$

Define $X$ with shape $(m, n^0)$ by stacking $x_1, x_2, \dots x_m$ as
$$
X = \begin{pmatrix}
- & x_1^T & -\\
- & x_2^T & -\\
&\vdots& \\
- & x_m^T & -
\end{pmatrix}	
$$

We define the network with $L$ layers where the $L$'th layer is the output layer and layer $0$ is the input layer. The layers $1, 2, \dots, L-1$ are known as the hidden layers. We define $n^l$ to be the number of nodes in layer $l$ for $l \in 0, 1, \dots, L$.

#### Feed forwards

For a fully connected network the input to a node is equal to the sum of the output from each of the nodes in the previous layer multiplied by the weights for that layer plus a bias.  
The output of the node is calculated by transforming the input using the activation function $g$. Various activation functions can be used but often it is chosen to be the sigmoid function, the relu function or the tanh function.

For node $i$ in layer $l$ where $i\in 1,\dots n^l$ we define the net input $z^l_i$ and output $a^l_i$ as follows

$$
\begin{align}
z^l_i&=\sum^{n^{l-1}}_{j=0}W^l_{ij}a^{l-1}_j + \beta^l_i \quad i\in 1,\dots, n^l\quad(1)\\
a^l_i&=g(z^l_i) \quad i\in 1,\dots, n^l\quad(2)\\
\end{align}
$$

Note weight $W_{ij}^l$ goes from $a_j^{l-1} \to z_i^{l}$

Let's assume that the activation function $g$ is the sigmoid function then 

$$
\begin{align}
g(x)&=\frac{1}{1+e^{-x}}\quad(3)\\
g'(x)&=g(x)(1-g(x))\quad(4)
\end{align}
$$

Feed forward as vectors

$$
\begin{align}
z^l &= W^l*a^{l-1} + \beta^l\quad(5)\\
a^l &= g(z^l)\quad(6)\\
\end{align}\\
\quad z^l,\beta^l\in \mathbb{R}^{(n^l, 1)}
\quad a^{l-1}\in \mathbb{R}^{(n^{l-1}, 1)}
\quad W^{l-1} \in \mathbb{R}^{(n^{l}, n^{l-1})}
$$

Feed forward as matrices! Performing on multiple samples $x_i$ at the same time

$$
\begin{align}
Z^l &= W^l*A^{l-1} + B^l\quad(7)\\
A^l &= g(Z^l)\quad(8)\\
Z^1 &= W^1*X^{T} + B^l\quad(9)\\
\end{align}\\
\quad Z^l, B^l\in \mathbb{R}^{(n^l, m)}
\quad a^{l-1}\in \mathbb{R}^{(n^{l-1}, m)}
\quad W^{l-1} \in \mathbb{R}^{(n^{l}, n^{l-1})}
$$

Note the columns of $Z^l$ and $A^l$ relate to $z^l$ and $a^l$  for the individual samples $x_k \quad k\in 1\dots m$

$$
Z^l = \begin{pmatrix}
| & | & \dots & |\\
z^l_{x_1} & z^l_{x_2} & \dots & z^l_{x_m}\\
| & | & \dots & |
\end{pmatrix}
\quad
A^l = \begin{pmatrix}
| & | & \dots & |\\
a^l_{x_1} & a^l_{x_2} & \dots & a^l_{x_m}\\
| & | & \dots & |
\end{pmatrix}	
$$

The m columns of $B^l$ are all copies of $\beta^l$

$$
B^l = \begin{pmatrix}
| & | & \dots & |\\
\beta^l & \beta^l & \dots & \beta^l\\
| & | & \dots & |
\end{pmatrix}	
$$

#### Back propagation

In the binary classification context we can define the loss a.k.a cost function as the cross entropy (for one input data item $x_i$ and true output $y_i$ where $a^L_1$ is the model prediction of $y_i$ being in class 1 or $P(y\in C_1)$ the loss is defined as follows:  

$$
\mathcal{L} = y_i\log(a^L_1)+(1-y_i)\log(1-a^L_{1})\quad(10)
$$

Note this is just the loss for one given sample $x_i$

As in all the other algorithms we seek to improve the model by changing the weights to reduce the loss. We do this by stepping the weights in the direction of the negative gradient of the cost function.

$$
\begin{align}
W^l_{ij} &\to W^l_{ij} + \eta \frac{\partial \mathcal{L}}{\partial W^l_{ij}} 
\quad i\in 1,\dots, n^l
\quad j\in 1,\dots, n^{l-1}
\quad l\in 1,\dots, L \quad(11)\\
\beta^l_{i} &\to \beta^l_{i} + \eta \frac{\partial \mathcal{L}}{\partial \beta^l_{i}} 
\quad i\in 1,\dots, n^l
\quad l\in 1,\dots, L\quad(12)
\end{align}
$$

Back propagation is involved but ultimately comes down to calculating $\frac{\partial \mathcal{L}}{\partial W^l_{ij}} $ and $\frac{\partial \mathcal{L}}{\partial \beta^l_{i}}$.  
As the name back propagation implies we calculate the derivatives starting at the end or the "right" of the network and propagate the error backwards (using the chain rule).

Above we have defined the loss for one training example. In this way we can updated the weights in a stochastic manner - or we can average the error across multiple samples and used batch gradient descent or just regular gradient descent if we average across all samples.

**Calculating $\frac{\partial \mathcal{L}}{\partial W^l_{ij}} $ and $\frac{\partial \mathcal{L}}{\partial \beta^l_{i}}$**

Note for each layer $l$

$$
\begin{align}
\frac{\partial \mathcal{L}}{\partial W^l_{ij}} &= 
\frac{\partial \mathcal{L}}{\partial z^l_{i}}
\frac{\partial z^l_{i}}{\partial W^l_{ij}}
, \quad
\frac{\partial \mathcal{L}}{\partial \beta^l_{i}} = 
\frac{\partial \mathcal{L}}{\partial z^l_{i}}
\frac{\partial z^l_{i}}{\partial \beta^l_{i}}\quad(13)
\end{align}
$$

or defining $\delta^l_i=\frac{\partial \mathcal{L}}{\partial z^l_{i}}$

$$
\begin{align}
\frac{\partial \mathcal{L}}{\partial W^l_{ij}} &= 
\delta^l_i
\frac{\partial z^l_{i}}{\partial W^l_{ij}}
, \quad
\frac{\partial \mathcal{L}}{\partial \beta^l_{i}} = 
\delta^l_i
\frac{\partial z^l_{i}}{\partial \beta^l_{i}}\quad(14)
\end{align}
$$

We know ( by differentiating equation (1))

$$
\frac{\partial z^l_{i}}{\partial W^l_{ij}} = a^{l-1}_j
,\quad
\frac{\partial z^l_{i}}{\partial \beta^l_{i}} = 1
\quad(15)
$$

So in order to calculate the derivatives we must calculate $\delta^l_i$

For the very last layer (when there is only one node in the output layer) we can use the chain rule to write

$$
\begin{align}
\frac{\partial \mathcal{L}}{\partial W^L_{1j}} &= 
\frac{\partial \mathcal{L}}{\partial z^L_{1}}
\frac{\partial z^L_{1}}{\partial W^L_{1j}} = 
\delta^L_1
\frac{\partial z^L_{1}}{\partial W^L_{1j}}\quad(16)\\
\delta^L_1 &= 
\frac{\partial \mathcal{L}}{\partial a^L_{1}}
\frac{\partial a^L_{1}}{\partial z^L_{1}}
\quad(17)\\
\end{align}
$$

Note

$$
\begin{align}
\frac{\partial \mathcal{L}}{\partial a^L_{1}} &= 
\frac{\partial}{\partial a^L_{1}}\left(
y_i\log(a^L_1)+(1-y_i)\log(1-a^L_{1})
\right)\quad(18)\\
\frac{\partial \mathcal{L}}{\partial a^L_{1}} &= 
\frac{y_i}{a^L_1} + \frac{1-y_i}{1-a^L_1}\quad(19)
\end{align}
$$

Assuming sigmoid activation

$$
\begin{align}
\frac{\partial a^L_{i}}{\partial z^L_{i}} &=
\frac{\partial}{\partial z^L_{i}}\left(
g(z^L_{i})
\right) = 
\frac{\partial}{\partial z^L_{i}}\left(
\sigma(z^L_{i})
\right)\\
&= \sigma'(z^L_{i})\\
&= \sigma(z^L_{i})(1-\sigma(z^L_{i}))\\
&= a^L_{i}*(1-a^L_{i})\quad(20)
\end{align}
$$

Based on equations (19) and (20)

$$
\delta^L_1 = a^L_1 - y\quad(21)
$$

Calculating $\delta^l_i$ for $l<L$

$$
\begin{align}
\delta^l_i &= \frac{\partial \mathcal{L}}{\partial z^l_{i}} \\
\delta^l_i &= \frac{\partial \mathcal{L}}{\partial a^l_{i}}
\frac{\partial a^l_{i}}{\partial z^l_{i}}\\
\delta^l_i &= \left(
\sum_{j=1}^{n^{l+1}}
\frac{\partial \mathcal{L}}{\partial z^{l+1}_{j}}
\frac{\partial z^{l+1}_{j}}{\partial a^l_{i}}
\right)
\frac{\partial a^l_{i}}{\partial z^l_{i}}\quad(22)
\end{align}
$$

Differentiating (1) to get $\frac{\partial z^{l+1}_{j}}{\partial a^l_{i}}$ and using equation (20) we can see

$$
\begin{align}
\delta^l_i &= \left(
\sum_{j=1}^{n^{l+1}}
\frac{\partial \mathcal{L}}{\partial z^{l+1}_{j}}
W_{ji}^{l+1}
\right)
a^l_{i}*(1-a^l_{i})\quad(23)
\end{align}
$$

And that's it.

Or we can do it again using vectors/matrices instead of the scalar derivation above. Equations (11) and (12) become

$$
\begin{align}
W^l &\to W^l - \eta \frac{\partial \mathcal{L}}{\partial W^l} 
\quad l\in 1,\dots, L \quad(24)\\
\beta^l &\to \beta^l - \eta \frac{\partial \mathcal{L}}{\partial \beta^l} 
\quad l\in 1,\dots, L\quad(25)
\end{align}
$$

Equation (23) becomes

$$
\begin{align}
\delta^l &= (W^{l+1})^T\delta^{l+1}\odot a^l\odot(1-a^l)\quad(26)
\end{align}
$$

Hence
$$
\frac{\partial \mathcal{L}}{\partial W^l}  = \delta^l(a^{l-1})^T\quad
\frac{\partial \mathcal{L}}{\partial \beta^l}  = \delta^l\quad(27)
$$

Back propagation as matrices! Performing on multiple samples $x_i$ at the same time using $X$. This is the batch approach to gradient descent.

If we define $Y$ with shape $(n^L, m) = (1, m)$ as 

$$
Y = \begin{pmatrix}
| & | & \dots & |\\
y_1 & y_2 & \dots & y_m\\
| & | & \dots & |
\end{pmatrix}	
$$

$$
\begin{align}
\mathcal{L} = \frac{1}{m}\sum_{k=1}^{m}Y_{1k}\log(A^L_{1k})+(1-Y_{1k})\log(1-A^L_{1k})
\quad(28)
\end{align}
$$

We define 
$$D^L = A^L - Y = 
\begin{pmatrix}
| & | & \dots & |\\
\delta^l_{x_1} & \delta^l_{x_2} & \dots & \delta^l_{x_m}\\
| & | & \dots & |
\end{pmatrix} \in \mathbb{R}^{(n^{L}, m)}
$$

$$
D^l = (W^{l+1})^TD^{l+1}\odot A^l\odot(\mathbf{1}^{(n^l,m)}-A^l)
\quad\in \mathbb{R}^{(n^{l}, m)} \quad l\in 1\dots L-1
\quad(29)
$$

Then the updates are 
$$
\begin{align}
W^l &\to W^l - \frac{\eta}{m}D^l(A^{l-1})^T\quad(30)\\
\beta^l &\to \beta^l - \frac{\eta}{m}D^l\mathbf{1}^{(m,1)}\quad(31)
\end{align}
$$

**Extending to regression and mulitclass classification**

In both of these cases we need to make two changes 
 1. We change the activation function in the final layer
 2. We change the cost function and so need to re-calculate $\delta^L$



**Regression**

1. We change the activation function in the final layer to simply be the identity 

$$
a^L_{1} = g(z^L_{1}) = z^L_{1} \implies \frac{\partial a^L_{1}}{\partial z^L_{1}} = 1
\quad(32)
$$

2. We define the cost function for one sample $x$ and corresponding $y$ as

$$
\mathcal{L} = \frac{1}{2}(y - a^L_{1})^2\quad(33)
$$

We now need to update update equation (17) using (32)

$$
\begin{align}
\delta^L_1 &= 
\frac{\partial \mathcal{L}}{\partial a^L_{1}}
\frac{\partial a^L_{1}}{\partial z^L_{1}}
\quad(34)\\
\end{align}
$$

$$
\delta^L_1 
= \frac{\partial a^L_{1}}{\partial z^L_{1}}
=\frac{\partial \mathcal{L}}{\partial a^L_{1}}
\quad(35)
$$  

$$
\delta^L_1 = \frac{\partial \mathcal{L}}{\partial z^L_{1}} = a^L_{1} - y\quad(36)
$$

Note this is the same as equation (21)

Using the matrices above
$$
\mathcal{L} = \frac{1}{2m}\sum_{k=1}^{m}(Y_{1k} - A^L_{1k})^2
$$

**Multiclass classification (number classes > 2)**

1. We change the activation function in the final layer to be the softmax function. Note that each sample $x_i$ has corresponding $y_i$ with shape $(n^L,1)$. Also note the number of classes is $n^L$.

$$
\begin{align}
a^L_i &= \sigma_i(z^L_1, z^L_2, \dots, z^L_{n^l})\\
&= \frac{e^{z^L_i}}{\sum_{r=1}^{n^L}e^{z^L_r}}
\end{align}
$$

2. Defining the cost function (for sample $x$ and corresponding $y$), and again we update equation (17)

$$
\mathcal{L} = -\sum_{r=1}^{n^L}y_r\log(a^L_r)
$$

hence substituting for $a^L_r$

$$
\begin{align}
\mathcal{L} 
&= -\sum_{r=1}^{n^L}y_r\log\left(
\frac{e^{z^L_r}}{\sum_{s=1}^{n^L}e^{z^L_s}}
\right)\\
&= -\sum_{r=1}^{n^L}
\left(
y_r\log(e^{z^L_r}) - y_r\log({\sum_{s=1}^{n^L}e^{z^L_s}})
\right)\\
&= -\sum_{r=1}^{n^L}y_r z^L_r - \log({\sum_{s=1}^{n^L}e^{z^L_s}})\\
&= \log({\sum_{s=1}^{n^L}e^{z^L_s}}) - \sum_{r=1}^{n^L}y_r z^L_r
\end{align}
$$

hence

$$
\delta^L_i 
= \frac{\partial \mathcal{L}}{\partial z^L_{1}}
= \frac{e^{z^L_i}}{\sum_{r=1}^{n^L}e^{z^L_r}} - y_i = a^L_i - y_i
$$

Note this is the same as equation (21) again!

Using the matrices above
$$
\mathcal{L} = -\frac{1}{2m}\sum_{k=1}^{m}\sum_{r=1}^{n^L}
Y_{rk}\log(A^L_{rk})
$$

### Define the neural network

In [2]:
# You can ignore the random logging metaclass

class LoggedClassMeta(type):
    def __init__(cls, name, bases, dct,**kwargs):
        #  print(cls, name, bases, dct,kwargs)
        if '_logging_level' in dct:
            level = dct['_logging_level']
        else:
            level = logging.INFO
        logger = logging.getLogger(name)
        logger.setLevel(level)
        cls.logger = logger

        
class LoggedClass(metaclass=LoggedClassMeta):
    pass


logging.basicConfig()


class NeuralNetwork(LoggedClass):
    _logging_level = logging.INFO
    
    def __init__(self,
                 layer_sizes=[5,10,1],
                 is_classifier=True,
                 learning_rate=0.1):
        self.layer_sizes = layer_sizes # n^0,...,n^L above
        self.is_classifier = is_classifier
        self.learning_rate = learning_rate  # eta above
        self.n_L = layer_sizes[-1]  # n^L above
        self.n_layers = len(layer_sizes) - 1  # L above
        self.initialise_weights()
        
    def initialise_weights(self):
        self.weight_matrices = [
            np.random.normal(loc=0.0, scale=1.0, size=(n_l, n_l_minus_1))
            for n_l, n_l_minus_1 in zip(self.layer_sizes[1:], self.layer_sizes)
        ]
        self.betas = [np.zeros(shape=(n_l, 1)) for n_l in self.layer_sizes[1:]]
    
    def feed_forward(self, X):
        m = X.shape[0]
        layer_activations = [X.T]
        for layer in range(self.n_layers):
            A_layer_minus_1 = layer_activations[-1]
            beta = self.betas[layer]
            B = np.repeat(beta, m, axis=-1)
            Z = self.weight_matrices[layer] @ A_layer_minus_1 + B
            A = self.activation_function(Z, layer=layer)
            layer_activations.append(A)
            self.log_layer(layer, A_layer_minus_1, beta, B, Z, A)
        self.layer_activations = layer_activations
        return layer_activations[-1]

    def back_propogation(self, X, Y):
        assert X.shape[0] == Y.shape[1]
        final_layer_error = self.layer_activations[-1] - Y
        D_plus_1 = final_layer_error
        # errors represent D matrices
        errors = [D_plus_1]
        for layer in range(self.n_layers - 2, -1, -1):
            self.logger.debug(f'Calculating D_{layer + 1}')
            A = self.layer_activations[layer + 1]
            self.log_back_prop_layer(layer, A, D_plus_1)
            D = (self.weight_matrices[layer + 1].T @ D_plus_1) * \
                A * (1 - A)
            D_plus_1 = D
            errors.insert(0, D)
        self.errors= errors
        self.update_weights()

    def update_weights(self):
        for layer in range(self.n_layers):
            m = self.errors[0].shape[1]
            d_L_d_W = (1 / m) * self.errors[layer] @ \
                self.layer_activations[layer].T
            d_L_d_beta = (1 / m) * self.errors[layer].sum(axis=1)[:, None]
            self.weight_matrices[layer] = self.weight_matrices[layer] - \
                self.learning_rate * d_L_d_W
            if layer==0:
                self.d_L_d_Ws.append(d_L_d_W.sum())
            self.betas[layer] = self.betas[layer] - \
                self.learning_rate * d_L_d_beta
            
    def log_layer(self, layer, A_layer_minus_1, beta, B, Z, A):
        self.logger.debug(
            f'A_layer_minus_1 i.e. A_{layer} '
            f'has shape {A_layer_minus_1.shape}')
        self.logger.debug(f'beta_{layer + 1} has shape {beta.shape}')
        self.logger.debug(f'B_{layer + 1} has shape {B.shape}')
        self.logger.debug(f'Z_{layer + 1} has shape {Z.shape}')
        self.logger.debug(f'A_{layer + 1} has shape {A.shape}')
    
    def log_back_prop_layer(self, layer,  A, D_plus_1):
        self.logger.debug(
            f'A_{layer + 1} has shape {A.shape}')
        self.logger.debug(
            f'W_{layer + 2} has shape '
            f'{self.weight_matrices[layer + 1].shape}')
        self.logger.debug(
            f'D_{layer + 2} has shape {D_plus_1.shape}')

    def activation_function(self, Z, layer):
        if layer == (self.n_layers -1):
            if not self.is_classifier:
                return Z
            if self.is_classifier and self.n_L >= 2:
                return np.exp(Z - logsumexp(Z, axis=0)[None,:])
        return expit(Z)
    
    def cost(self, Y):
        if self.is_classifier and self.n_L == 1:
            cost = (-1 / m) * (
                Y * np.log(self.layer_activations[-1]) + \
                (1 - Y) * np.log(1 - self.layer_activations[-1])
            ).sum()
        if self.is_classifier and self.n_L > 1:
            cost = (-1 / m) * \
                (Y * np.log(self.layer_activations[-1])).sum()
        if not self.is_classifier:
            cost = (1 / (2 * m)) * \
                ((Y - self.layer_activations[-1]) ** 2).sum()
        self.logger.debug(f'cost = {cost}')
        self.costs.append(cost)
    
    def fit(self, X, Y, epochs=100):
        if self.n_L > 1:
            if Y.shape[0] != self.n_L:
                print('One hot encoding Y')
                Y = np.eye(self.n_L)[:,Y.reshape(-1).astype(int)]
        self.costs = []
        self.d_L_d_Ws = []
        for epoch in range(epochs):
            self.feed_forward(X)
            self.cost(Y)
            self.back_propogation(X, Y)
    
    def predict(self, X):
        A_L = self.feed_forward(X)
        if not self.is_classifier:
            return A_L
        if self.is_classifier and self.n_L == 1:
            return np.round(A_L).astype(int)
        if self.is_classifier and self.n_L > 1:
            return np.argmax(A_L, axis=0)
    
    def predict_proba(self, X):
        A_L = self.feed_forward(X)
        if not self.is_classifier:
            raise Exception('Must be a classifier')
        return A_L
            

## Neural network on toy example

### Make fake data

In [3]:
m = 20
X = np.linspace(0, 10, m).reshape((m, 1))
epsilon =  np.random.normal(scale=2, size=(m, 1))
Y = X ** 2 + X + epsilon
Y = (Y > 30).astype(int).T

### Fit network and visualise cost

In [4]:
neural_network = NeuralNetwork(
    layer_sizes=[1,3,1],
    learning_rate=0.3)
neural_network.fit(X, Y, epochs=500)

In [5]:
fig = go.FigureWidget()
fig.add_scatter(
    x=list(range(len(neural_network.costs))),
    y=neural_network.costs)
fig

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': '02077f81-e63e-4370-9ad3-9c28a97c3ef8',
 …

In [6]:
for i, (val,cost) in enumerate(zip(neural_network.d_L_d_Ws,neural_network.costs)):
    if i % 100 == 0:
        print(f'iteration {i} with val {abs(val):.3f}, cost: {cost:.3f}')

iteration 0 with val 0.247, cost: 0.730
iteration 100 with val 0.029, cost: 0.372
iteration 200 with val 0.001, cost: 0.206
iteration 300 with val 0.001, cost: 0.142
iteration 400 with val 0.001, cost: 0.110


In [7]:
Y

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [8]:
neural_network.predict_proba(X)
neural_network.predict(X)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

## Neural network binary classifier - Titanic data

### Load titanic data set

In [9]:
X_train = pd.read_feather('../titanic/processed/X_train.feather')
y_train = pd.read_feather('../titanic/processed/y_train.feather')
X_test = pd.read_feather('../titanic/processed/X_test.feather')
y_test = pd.read_feather('../titanic/processed/y_test.feather')

### Neural network model accuracy

In [10]:
titanic_nn = NeuralNetwork(layer_sizes=[30,50,1], learning_rate=0.5)
titanic_nn.fit(X_train.values, y_train.values.T, epochs=400)
y_pred = titanic_nn.predict(X_test.values)
test_acc = (y_pred == y_test.values.flatten()).sum() / len(y_test)
print(f'Test accuracy = {test_acc:.2%}')

Test accuracy = 78.77%


In [11]:
fig = go.FigureWidget()
fig.add_scatter(
    x=list(range(len(titanic_nn.costs))),
    y=titanic_nn.costs)
fig

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': '09a1b12b-8192-4c46-90f1-7af1496cf419',
 …

## Neural network classifier - Iris data

### Load the iris data set

In [12]:
iris_data = load_iris()
iris_df = pd.DataFrame(iris_data['data'],columns=iris_data['feature_names'])
iris_df['y'] = iris_data['target']
iris_df = iris_df.sample(frac=1, random_state=42).reset_index(drop=True)
iris_sample = iris_df.head(5)
iris_sample

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),y
0,6.1,2.8,4.7,1.2,1
1,5.7,3.8,1.7,0.3,0
2,7.7,2.6,6.9,2.3,2
3,6.0,2.9,4.5,1.5,1
4,6.8,2.8,4.8,1.4,1


### Fit neural network classifier and visualise

In [13]:
iris_sample_vals = iris_sample.values

# # for small sample
# X = iris_sample_vals[:,:-1]
# y = iris_sample_vals[:,-1]

X = iris_df.values[:,:-1]
m = X.shape[0]
y = iris_df.values[:,-1].reshape(1, m)

iris_neural_network = NeuralNetwork(layer_sizes=[4,8,3])
iris_neural_network.fit(X, y, epochs=800)

feature_names = iris_data['feature_names']

One hot encoding Y


In [14]:
fig = go.FigureWidget()
fig.add_scatter(
    x=list(range(len(iris_neural_network.costs))),
    y=iris_neural_network.costs)
fig

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': 'c9bca653-9f6f-4b44-875c-1843768321cf',
 …

### Example prediction on Iris data set

In [15]:
iris_neural_network.predict(X[:4,:])

array([1, 0, 2, 1])

In [16]:
y[:,:4]

array([[1., 0., 2., 1.]])

In [17]:
iris_neural_network.predict_proba(X)[:,:4]

array([[0.0244518 , 0.96999122, 0.00134162, 0.03084734],
       [0.80990323, 0.02856427, 0.04705977, 0.75291137],
       [0.16564497, 0.00144451, 0.95159861, 0.2162413 ]])

In [18]:
iris_neural_network.predict(X)

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0,
       0, 1, 2, 2, 1, 2, 1, 2, 1, 0, 2, 1, 0, 0, 0, 1, 2, 0, 0, 0, 1, 0,
       1, 2, 0, 1, 2, 0, 2, 2, 1, 2, 2, 1, 0, 1, 2, 0, 0, 1, 1, 0, 2, 0,
       0, 2, 1, 2, 2, 2, 2, 1, 0, 0, 2, 2, 0, 0, 0, 1, 2, 0, 2, 2, 0, 1,
       1, 2, 1, 2, 0, 2, 1, 2, 1, 1, 1, 0, 1, 1, 0, 1, 2, 2, 0, 1, 2, 2,
       0, 2, 0, 1, 2, 2, 1, 2, 1, 1, 2, 2, 0, 1, 2, 0, 1, 2])

In [19]:
y.astype(int)

array([[1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
        0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0,
        0, 1, 2, 2, 1, 2, 1, 2, 1, 0, 2, 1, 0, 0, 0, 1, 2, 0, 0, 0, 1, 0,
        1, 2, 0, 1, 2, 0, 2, 2, 1, 1, 2, 1, 0, 1, 2, 0, 0, 1, 1, 0, 2, 0,
        0, 1, 1, 2, 1, 2, 2, 1, 0, 0, 2, 2, 0, 0, 0, 1, 2, 0, 2, 2, 0, 1,
        1, 2, 1, 2, 0, 2, 1, 2, 1, 1, 1, 0, 1, 1, 0, 1, 2, 2, 0, 1, 2, 2,
        0, 2, 0, 1, 2, 2, 1, 2, 1, 1, 2, 2, 0, 1, 2, 0, 1, 2]])

In [20]:
(iris_neural_network.predict(X) == y.astype(int)).sum() / y.shape[1]
# Likely overfitting as no regularization!

0.98

## Neural network regressor - Boston housing data

### Load Boston housing data

In [21]:
boston_data = load_boston()
boston_df = pd.DataFrame(boston_data['data'], columns=boston_data['feature_names'])
boston_df['y'] = boston_data['target']
boston_df = boston_df.sample(frac=1, random_state=42).reset_index(drop=True)
boston_sample = boston_df.head(5)
boston_sample

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,y
0,0.09178,0.0,4.05,0.0,0.510,6.416,84.1,2.6463,5.0,296.0,16.6,395.50,9.04,23.6
1,0.05644,40.0,6.41,1.0,0.447,6.758,32.9,4.0776,4.0,254.0,17.6,396.90,3.53,32.4
2,0.10574,0.0,27.74,0.0,0.609,5.983,98.8,1.8681,4.0,711.0,20.1,390.11,18.07,13.6
3,0.09164,0.0,10.81,0.0,0.413,6.065,7.8,5.2873,4.0,305.0,19.2,390.91,5.52,22.8
4,5.09017,0.0,18.10,0.0,0.713,6.297,91.8,2.3682,24.0,666.0,20.2,385.09,17.27,16.1


### Fit neural network on Boston data

In [22]:
X = boston_df.values[:,:-1]
m = X.shape[0]
y = boston_df.values[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
standard_scaler = StandardScaler()
X_train = standard_scaler.fit_transform(X_train)
X_test = standard_scaler.transform(X_test)
y = y.reshape(1, y.shape[0])
y_train = y_train.reshape(1, y_train.shape[0])
y_test = y_test.reshape(1, y_test.shape[0])

In [23]:
X_train.shape

(404, 13)

In [24]:
boston_neural_network = NeuralNetwork(
    layer_sizes=[13,15,1],
    learning_rate=0.2,
    is_classifier=False)

boston_neural_network.fit(X_train, y_train, epochs=1000)

### Plotting Boston neural network error

In [25]:
fig = go.FigureWidget()
fig.add_scatter(
    x=list(range(len(boston_neural_network.costs))),
    y=boston_neural_network.costs)
fig

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': '6abb282c-bfe9-42dd-9a09-8f697caf822a',
 …

### Neural Network accuracy on Boston data

In [26]:
y_pred = boston_neural_network.predict(X_test)
test_acc = r2_score(y_test[0], y_pred[0])
print(f'Test accuracy (R2 score) = {test_acc:.2%}')

Test accuracy (R2 score) = 87.17%


In [27]:
y_pred[:,:5]

array([[24.92937298, 20.00371964, 23.04416689, 21.03451595, 17.72778921]])

In [28]:
y_test[:,:5]

array([[25.2, 19.3, 20.5, 21.2, 21.8]])

## Neural network classifier - mnist data 

### load mnist data

In [174]:
files = [
    'train-images-idx3-ubyte.gz',
    'train-labels-idx1-ubyte.gz',
    't10k-images-idx3-ubyte.gz',
    't10k-labels-idx1-ubyte.gz']

path='../mnist'
path = pathlib.Path(path)
    
def download_mnist(path):
    path.mkdir(exist_ok=True)
    url = 'http://yann.lecun.com/exdb/mnist/'
    for file in files:
        if file not in [file.name for file in path.iterdir()]:
            urlretrieve(url + file, path /  file)
            print(f"Downloaded {file} to {path}")

def images(path):
    """Return images loaded locally."""
    with gzip.open(path) as f:
        pixels = np.frombuffer(f.read(), 'B', offset=16)
    return pixels.reshape(-1, 784).astype('float32') / 255

def labels(path):
    """Return labels loaded locally."""
    with gzip.open(path) as f:
        # First 8 bytes are magic_number, n_labels
        integer_labels = np.frombuffer(f.read(), 'B', offset=8)
    return integer_labels

In [197]:
X_train = images(path / files[0])
y_train = labels(path / files[1])
X_test = images(path / files[2])
y_test = labels(path / files[3])

### Visualise mnist samples

In [198]:
from itertools import product

def display_sample(sample_id, X_train, y_train):
    pixels = 28
    sample = X_train[sample_id].reshape((pixels,pixels))
    y = y_train[sample_id]
    print(y)
    fig = go.FigureWidget()
    fig.add_heatmap(
        z=sample,
        showscale=False,
        colorscale=[
            [0, "rgb(255, 255, 255)"],
            [1.0, "rgb(0, 0, 0)"]])
    fig.layout.yaxis.autorange = "reversed"
    fig.layout.xaxis.visible=False
    fig.layout.yaxis.visible=False
    fig.layout.title = f'Sample {y} of digits'
    return fig


def display_samples(X_train,
                   n_rows = 5,
                   n_columns = 10):
    pixels = 28
    samples = np.ones(shape=(pixels * n_rows, pixels * n_columns))
    for i, (row, col) in enumerate(product(range(n_rows),range(n_columns))):
        samples[row*28:(row+1)*28,col*28:(col+1)*28] = \
            X_train[i].reshape(pixels, pixels)
    fig = go.FigureWidget()
    fig.add_heatmap(
        z=samples,
        showscale=False,
        colorscale=[
            [0, "rgb(255, 255, 255)"],
            [1.0, "rgb(0, 0, 0)"]])
    fig.layout.yaxis.autorange = "reversed"
    fig.layout.xaxis.visible=False
    fig.layout.yaxis.visible=False
    fig.layout.title = 'Sample of digits'
    return fig

In [200]:
display_sample(4, X_train, y_train)

9


FigureWidget({
    'data': [{'colorscale': [[0, 'rgb(255, 255, 255)'], [1.0, 'rgb(0, 0, 0)']],
              '…

In [201]:
display_samples(X_train)

FigureWidget({
    'data': [{'colorscale': [[0, 'rgb(255, 255, 255)'], [1.0, 'rgb(0, 0, 0)']],
              '…

### Fit neural network to mnist data

In [252]:
mnist_network = NeuralNetwork(
    layer_sizes=[28*28,400,10],
    learning_rate=0.5)

In [253]:
mnist_network.fit(X_train, y_train, epochs=200)

One hot encoding Y


### Plot the cost against epochs 

In [255]:
fig = go.FigureWidget()
fig.add_scatter(
    x=list(range(len(mnist_network.costs))),
    y=mnist_network.costs)
fig

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': 'e9ea711c-415e-4bfc-93f1-9b943633868a',
 …

### Neural network accuracy

In [256]:
train_pred = mnist_network.predict(X_train)
train_acc = (train_pred == y_train).sum() / y_train.shape[0]
train_acc

0.8462166666666666

In [257]:
test_pred = mnist_network.predict(X_test)
test_acc = (test_pred == y_test).sum() / y_test.shape[0]
test_acc

0.8411

## end